In [12]:
import csv
import pandas as pd
from typing import *

from typing import Tuple

In [15]:
def read_csv(filename: str, **csv_options) -> List[List[str]]:
    with open(filename, 'r') as fd:
        reader = csv.reader(fd, **csv_options)
        data_read = [row for row in reader]

    return data_read


def write_csv(filename: str, data: List[List[Any]], **csv_options) -> None:
    with open(filename, 'wt') as fd:
        writer = csv.writer(fd, **csv_options)
        writer.writerows(data)


def split_df(df: List[List[str]]) -> Tuple[Dict[str, int], List[List[str]]]:
    header = dict(zip(df[0], range(len(df[0]))))
    df = df[1:]

    return header, df


def convert_types(df: List[List[str]], type_arr: Tuple[TypeVar]) -> None:
    for df_row in df:
        for i in range(len(df_row)):
            df_row[i] = type_arr[i](df_row[i])


def get_department_structure(df: List[List[Any]], header: Dict[str, int]) -> Dict[str, Set[str]]:
    departments_and_teams = dict()
    dep_ind = header['Департамент']
    team_ind = header['Отдел']

    for df_row in df:
        department = df_row[dep_ind]
        team = df_row[team_ind]

        if departments_and_teams.get(department) is None:
            departments_and_teams[department] = set()

        if team not in departments_and_teams[department]:
            departments_and_teams[department].add(team)

    return departments_and_teams


def print_department_structure(dep_structure: Dict[str, Set[str]]) -> None:
    for department, teams in dep_structure.items():
        print(department, end=': ')

        for i, team in enumerate(teams):
            if i == 0:
                print(team)
            else:
                print(' ' * (len(department)+1), team)


def get_department_report(df: List[List[Any]], header: Dict[str, int]) -> Dict[str, Dict[str, int]]:
    departments_and_info = dict()
    dep_ind = header['Департамент']
    salary_ind = header['Оклад']
    def_info = {'count': 0,
                'min': float('inf'),
                'max': float('-inf'),
                'avg': 0}

    for df_row in df:
        department = df_row[dep_ind]
        salary = df_row[salary_ind]

        if departments_and_info.get(department) is None:
            departments_and_info[department] = def_info.copy()

        departments_and_info[department]['count'] += 1
        departments_and_info[department]['avg'] += salary
        departments_and_info[department]['min'] = \
            min(salary, departments_and_info[department]['min'])
        departments_and_info[department]['max'] = \
            max(salary, departments_and_info[department]['max'])

    for department in departments_and_info.keys():
        dep_size = departments_and_info[department]['count']
        departments_and_info[department]['avg'] /= dep_size

    return departments_and_info


def print_department_report(report: Dict[str, Dict[str, int]]) -> None:
    if len(report) == 0:
        return

    header = list(report.values())[0].keys()
    header_format_str = '{:<32} {:<8} {:<16} {:<16} {:<16}'
    format_str = '{:<32} {:<8} {:<16} {:<16} {:.1f}'

    print(header_format_str.format('Department', *header))
    for dep, info in report.items():
        info = info.values()
        print(format_str.format(dep, *info))


def save_report(report: Dict[str, Dict[str, int]], filename: str, **csv_options) -> None:
    if len(report) == 0:
        return

    header = list(report.values())[0].keys()
    data_to_save = [list(header)]

    for dep, info in report.items():
        df_row = [dep] + list(info.values())
        data_to_save.append(df_row)

    write_csv(filename, data_to_save, **csv_options)

In [16]:
filename = 'Corp_Summary.csv'
df = read_csv(filename, delimiter=';')
header, df = split_df(df)
convert_types(df, (str, str, str, str, float, int))

In [31]:
departments_and_teams = dict()
dep_ind = header['Департамент']
team_ind = header['Отдел']

for df_row in df:
    department = df_row[dep_ind]
    team = df_row[team_ind]

    if departments_and_teams.get(department) is None:
        departments_and_teams[department] = set()

    if team not in departments_and_teams[department]:
        departments_and_teams[department].add(team)

for department, teams in departments_and_teams.items():
    print(department, end=': ')

    for i, team in enumerate(teams):
        if i == 0:
            print(team)
        else:
            print(' ' * (len(department)+1), team)

Разработка: Внутренний портал
            Основной продукт
            Платформа
Маркетинг: Performance
           Direct
Бухгалтерия: Компенсации и льготы
             Зарплата
Аналитика: DWH
           Product
Продажи: Госы
         B2B
         B2C


In [33]:
# Dict[str, Dict[str, int]]
departments_and_info = dict()
dep_ind = header['Департамент']
salary_ind = header['Оклад']
def_info = {'count': 0,
            'min': float('inf'),
            'max': float('-inf'),
            'avg': 0}

for df_row in df:
    department = df_row[dep_ind]
    salary = df_row[salary_ind]

    if departments_and_info.get(department) is None:
        departments_and_info[department] = def_info.copy()

    departments_and_info[department]['count'] += 1
    departments_and_info[department]['avg'] += salary
    departments_and_info[department]['min'] = \
        min(salary, departments_and_info[department]['min'])
    departments_and_info[department]['max'] = \
        max(salary, departments_and_info[department]['max'])

for department in departments_and_info.keys():
    dep_size = departments_and_info[department]['count']
    departments_and_info[department]['avg'] /= dep_size

print(departments_and_info)

{'Разработка': {'count': 47, 'min': 56000, 'max': 124900, 'avg': 91891.48936170213}, 'Маркетинг': {'count': 38, 'min': 55500, 'max': 124300, 'avg': 94202.63157894737}, 'Бухгалтерия': {'count': 41, 'min': 56100, 'max': 124900, 'avg': 92941.46341463414}, 'Аналитика': {'count': 34, 'min': 55200, 'max': 122800, 'avg': 89008.82352941176}, 'Продажи': {'count': 40, 'min': 55700, 'max': 122100, 'avg': 89307.5}}


In [51]:
report = departments_and_info
header = list(report.values())[0].keys()
header_format_str = '{:<32} {:<8} {:<16} {:<16} {:<16}'
format_str = '{:<32} {:<8} {:<16} {:<16} {:.1f}'

print(header_format_str.format('Department', *header))
for dep, info in report.items():
    info = info.values()
    print(format_str.format(dep, *info))

Department                       count    min              max              avg             
Разработка                       47       56000            124900           91891.5
Маркетинг                        38       55500            124300           94202.6
Бухгалтерия                      41       56100            124900           92941.5
Аналитика                        34       55200            122800           89008.8
Продажи                          40       55700            122100           89307.5
